<a href="https://colab.research.google.com/github/rahiakela/python-for-data-analysis/blob/7-data-cleaning-and-preparation/1_handling_missing_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Handling Missing Data

During the course of doing data analysis and modeling, a significant amount of time is spent on data preparation: loading, cleaning, transforming, and rearranging. Such tasks are often reported to take up 80% or more of an analyst’s time. Sometimes the way that data is stored in files or databases is not in the right format for a particular task. 

Many researchers choose to do ad hoc processing of data from one form to another using a general-purpose programming language like Python, Perl, R, or Java, or Unix text-processing tools like sed or awk. Fortunately, pandas, along with the built-in Python language features, provides you with a high-level, flexible, and fast set of tools to enable you to manipulate data into the right form.

The type of data manipulation includes:-
* Handling Missing Data
* Data Transformations
* String Manipulation

## Handling Missing Data

Missing data occurs commonly in many data analysis applications. One of the goals of pandas is to make working with missing data as painless as possible. For example, all of the descriptive statistics on pandas objects exclude missing data by default.

The way that missing data is represented in pandas objects is somewhat imperfect, but it is functional for a lot of users. For numeric data, pandas uses the floating-point value NaN (Not a Number) to represent missing data. We call this a sentinel value that can be easily detected.

In [0]:
import pandas as pd
import numpy as np

In [2]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])
string_data

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [3]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

In [4]:
string_data[0] = None
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

There is work ongoing in the pandas project to improve the internal details of how missing data is handled, but the user API functions, like pandas.isnull, abstract away many of the annoying details.

NA handling methods-
* **dropna**:Filter axis labels based on whether values for each label have missing data, with varying thresholds for how
much missing data to tolerate.
* **fillna**:Fill in missing data with some value or using an interpolation method such as 'ffill' or 'bfill'.
* **isnull**:Return boolean values indicating which values are missing/NA.
* **notnull**:Negation of isnull.

## Filtering Out Missing Data

There are a few ways to filter out missing data. While you always have the option to do it by hand using pandas.isnull and boolean indexing, the dropna can be helpful.

In [0]:
from numpy import nan as NA

In [6]:
data = pd.Series([1, NA, 3.5, NA, 7])
data

0    1.0
1    NaN
2    3.5
3    NaN
4    7.0
dtype: float64

In [7]:
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

This is equivalent to:

In [8]:
data = pd.Series([1, NA, 3.5, NA, 7])
data

0    1.0
1    NaN
2    3.5
3    NaN
4    7.0
dtype: float64

In [10]:
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

With DataFrame objects, things are a bit more complex. You may want to drop rows or columns that are all NA or only those containing any NAs. dropna by default drops any row containing a missing value:

In [11]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA], [NA, NA, NA], [NA, 6.5, 3.]])
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [12]:
# drops any row containing a missing value
cleaned = data.dropna()
cleaned

,0,1,2
0,1.0,6.5,3.0


In [15]:
# Passing how='all' will only drop rows that are all NA
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


In [16]:
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [19]:
# To drop columns in the same way, pass axis=1
data[4] = NA  # add col 4
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [20]:
data.dropna(how='all', axis=1) # drop column that contains all NA

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [21]:
data.dropna(how='all', axis=0) # drop row that contains all NA

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


A related way to filter out DataFrame rows tends to concern time series data. Suppose you want to keep only rows containing a certain number of observations. You can indicate this with the thresh argument:

In [22]:
df = pd.DataFrame(np.random.randn(7, 3))
df

,0,1,2
0,0.880431,-0.070019,-0.282181
1,1.007405,-0.243568,0.297359
2,-0.098472,-1.576629,-1.252267
3,-0.117484,1.657300,1.204843
4,-1.052626,0.030401,-0.009309
5,1.632493,0.749160,-0.034033
6,0.797872,-0.101625,-0.699033


In [23]:
df.iloc[:4, 1] = NA  # assign NA to second col from row 0 to 3rd
df

,0,1,2
0,0.880431,NaN,-0.282181
1,1.007405,NaN,0.297359
2,-0.098472,NaN,-1.252267
3,-0.117484,NaN,1.204843
4,-1.052626,0.030401,-0.009309
5,1.632493,0.749160,-0.034033
6,0.797872,-0.101625,-0.699033


In [25]:
df.iloc[:2, 2]= NA  # # assign NA to 3rd col from row 0 to second
df

,0,1,2
0,0.880431,NaN,NaN
1,1.007405,NaN,NaN
2,-0.098472,NaN,-1.252267
3,-0.117484,NaN,1.204843
4,-1.052626,0.030401,-0.009309
5,1.632493,0.749160,-0.034033
6,0.797872,-0.101625,-0.699033


In [26]:
# drops any row containing a missing value
df.dropna()

,0,1,2
4,-1.052626,0.030401,-0.009309
5,1.632493,0.749160,-0.034033
6,0.797872,-0.101625,-0.699033


In [27]:
df

,0,1,2
0,0.880431,NaN,NaN
1,1.007405,NaN,NaN
2,-0.098472,NaN,-1.252267
3,-0.117484,NaN,1.204843
4,-1.052626,0.030401,-0.009309
5,1.632493,0.749160,-0.034033
6,0.797872,-0.101625,-0.699033


In [31]:
# keep only rows containing a certain number of observations
df.dropna(thresh=2)

,0,1,2
2,-0.098472,NaN,-1.252267
3,-0.117484,NaN,1.204843
4,-1.052626,0.030401,-0.009309
5,1.632493,0.749160,-0.034033
6,0.797872,-0.101625,-0.699033


In [32]:
df.dropna(thresh=1)

,0,1,2
0,0.880431,NaN,NaN
1,1.007405,NaN,NaN
2,-0.098472,NaN,-1.252267
3,-0.117484,NaN,1.204843
4,-1.052626,0.030401,-0.009309
5,1.632493,0.749160,-0.034033
6,0.797872,-0.101625,-0.699033


In [33]:
df.dropna(thresh=3)

,0,1,2
4,-1.052626,0.030401,-0.009309
5,1.632493,0.749160,-0.034033
6,0.797872,-0.101625,-0.699033


## Filling In Missing Data

Rather than filtering out missing data (and potentially discarding other data along with it), you may want to fill in the “holes” in any number of ways. 

For most purposes, the fillna method is the workhorse function to use.Calling fillna with a constant replaces missing values with that value:

In [34]:
df.fillna(0)  # fill all NA with 0

,0,1,2
0,0.880431,0.000000,0.000000
1,1.007405,0.000000,0.000000
2,-0.098472,0.000000,-1.252267
3,-0.117484,0.000000,1.204843
4,-1.052626,0.030401,-0.009309
5,1.632493,0.749160,-0.034033
6,0.797872,-0.101625,-0.699033


In [35]:
# Calling fillna with a dict, you can use a different fill value for each column
df.fillna({1: 0.5, 2: 0})

,0,1,2
0,0.880431,0.500000,0.000000
1,1.007405,0.500000,0.000000
2,-0.098472,0.500000,-1.252267
3,-0.117484,0.500000,1.204843
4,-1.052626,0.030401,-0.009309
5,1.632493,0.749160,-0.034033
6,0.797872,-0.101625,-0.699033


In [0]:
# fillna returns a new object, but you can modify the existing object in-place
_ = df.fillna(0, inplace=True)

In [37]:
df

,0,1,2
0,0.880431,0.000000,0.000000
1,1.007405,0.000000,0.000000
2,-0.098472,0.000000,-1.252267
3,-0.117484,0.000000,1.204843
4,-1.052626,0.030401,-0.009309
5,1.632493,0.749160,-0.034033
6,0.797872,-0.101625,-0.699033


In [39]:
# The same interpolation methods available for reindexing can be used with fillna
df = pd.DataFrame(np.random.randn(6, 3))
df

,0,1,2
0,0.106127,-2.256435,-0.403545
1,1.378141,-0.348674,-0.436012
2,-0.163887,0.808093,-1.710423
3,-0.649799,0.351407,0.348341
4,0.752599,0.842303,-0.441815
5,-0.586780,0.467400,-1.739370


In [40]:
df.iloc[2:, 1] = NA  # assign NA to second col from row 3rd to 6th
df

,0,1,2
0,0.106127,-2.256435,-0.403545
1,1.378141,-0.348674,-0.436012
2,-0.163887,NaN,-1.710423
3,-0.649799,NaN,0.348341
4,0.752599,NaN,-0.441815
5,-0.586780,NaN,-1.739370


In [41]:
df.iloc[4:, 2] = NA  # assign NA to 3rd col from row 5th to 6th
df

,0,1,2
0,0.106127,-2.256435,-0.403545
1,1.378141,-0.348674,-0.436012
2,-0.163887,NaN,-1.710423
3,-0.649799,NaN,0.348341
4,0.752599,NaN,NaN
5,-0.586780,NaN,NaN


In [42]:
df.fillna(method='ffill') # fill NA with the same closed value

,0,1,2
0,0.106127,-2.256435,-0.403545
1,1.378141,-0.348674,-0.436012
2,-0.163887,-0.348674,-1.710423
3,-0.649799,-0.348674,0.348341
4,0.752599,-0.348674,0.348341
5,-0.586780,-0.348674,0.348341


In [44]:
df.fillna(method='ffill', limit=2)

,0,1,2
0,0.106127,-2.256435,-0.403545
1,1.378141,-0.348674,-0.436012
2,-0.163887,-0.348674,-1.710423
3,-0.649799,-0.348674,0.348341
4,0.752599,NaN,0.348341
5,-0.586780,NaN,0.348341


In [45]:
df.fillna(method='ffill', limit=1)

,0,1,2
0,0.106127,-2.256435,-0.403545
1,1.378141,-0.348674,-0.436012
2,-0.163887,-0.348674,-1.710423
3,-0.649799,NaN,0.348341
4,0.752599,NaN,0.348341
5,-0.586780,NaN,NaN


In [46]:
df.fillna(method='ffill', limit=3)

,0,1,2
0,0.106127,-2.256435,-0.403545
1,1.378141,-0.348674,-0.436012
2,-0.163887,-0.348674,-1.710423
3,-0.649799,-0.348674,0.348341
4,0.752599,-0.348674,0.348341
5,-0.586780,NaN,0.348341


With fillna you can do lots of other things with a little creativity. 
For example, you might pass the mean or median value.

In [47]:
df

,0,1,2
0,0.106127,-2.256435,-0.403545
1,1.378141,-0.348674,-0.436012
2,-0.163887,NaN,-1.710423
3,-0.649799,NaN,0.348341
4,0.752599,NaN,NaN
5,-0.586780,NaN,NaN


In [48]:
df.fillna(df.mean())

,0,1,2
0,0.106127,-2.256435,-0.403545
1,1.378141,-0.348674,-0.436012
2,-0.163887,-1.302554,-1.710423
3,-0.649799,-1.302554,0.348341
4,0.752599,-1.302554,-0.550410
5,-0.586780,-1.302554,-0.550410


In [49]:
df

,0,1,2
0,0.106127,-2.256435,-0.403545
1,1.378141,-0.348674,-0.436012
2,-0.163887,NaN,-1.710423
3,-0.649799,NaN,0.348341
4,0.752599,NaN,NaN
5,-0.586780,NaN,NaN


In [50]:
df.fillna(df.median())

,0,1,2
0,0.106127,-2.256435,-0.403545
1,1.378141,-0.348674,-0.436012
2,-0.163887,-1.302554,-1.710423
3,-0.649799,-1.302554,0.348341
4,0.752599,-1.302554,-0.419779
5,-0.586780,-1.302554,-0.419779


fillna function arguments:

* **value**:Scalar value or dict-like object to use to fill missing values
* **method**:Interpolation; by default 'ffill' if function called with no other arguments
* **axis**:Axis to fill on; default axis=0
* **inplace**:Modify the calling object without producing a copy
* **limit**:For forward and backward filling, maximum number of consecutive periods to fill